# 03 — Baseline Backtest (Opening Range Strategy)

**Goal of Notebook 3:** turn the audited days from `valid_days.csv` into a clean, reproducible **baseline backtest**:
- One decision per day at **10:22** based on **Opening Range 09:30–10:00 (inclusive)**:
  - **Long** if price is in the **top 35%** of OR
  - **Short** if in the **bottom 35%**
  - **No trade** otherwise
- **Risk:** SL = 25 pts, TP = 75 pts, **forced exit at 12:00**.
- **One trade max per day**.
- Compute P&L in **points**, then convert to **$** via config `point_value_usd`.  
- No slippage/fees at first (we’ll add them later as toggles).

## 3.1 — Baseline Backtest: scope & inputs (read-first)

**This section (3.1) just:**
1) Loads config + `valid_days.csv` (from Notebook 2).  
2) Prepares a map from **year → raw CSV path** to quickly fetch minute bars per day.  
3) Shows a tiny summary so we know how many days will be simulated.

Next section (3.2): we’ll implement a **small loader** that returns the **09:30–12:00** minute bars for a single date, ready for the trade logic.


In [1]:
# 3.1 — Load config, audited days, and wire up file references

from pathlib import Path
import pandas as pd
import yaml

# --- Paths (works whether you're inside notebooks/ or repo root) ---
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
CONFIG_DIR = ROOT / "config"
DATA_RAW_DIR = ROOT / "data" / "raw"
REPORTS_TBLS = ROOT / "reports" / "tables"

# --- Configs (single source of truth) ---
def load_yaml(p: Path):
    with open(p, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)

STRATEGY = load_yaml(CONFIG_DIR / "strategy.yml")
INSTR    = load_yaml(CONFIG_DIR / "instruments.yml")

# Frequently-used fields
session   = INSTR.get("session", {})
market    = INSTR.get("market", {})
costs     = INSTR.get("costs", {})
data_cfg  = INSTR.get("data", {})
params    = STRATEGY.get("parameters", {})

OR_START  = session.get("or_window", {}).get("start", "09:30")
OR_END    = session.get("or_window", {}).get("end_inclusive", "10:00")  # inclusive
ENTRY_T   = session.get("entry_time", "10:22")
EXIT_T    = session.get("hard_exit_time", "12:00")

TOP_PCT   = params.get("zones", {}).get("top_pct", 0.35)
BOT_PCT   = params.get("zones", {}).get("bottom_pct", 0.35)
SL_PTS    = params.get("risk", {}).get("stop_loss_points", 25)
TP_PTS    = params.get("risk", {}).get("take_profit_points", 75)
POINT_VAL = market.get("point_value_usd", 80.0)

# --- Audited trading days from Notebook 2 ---
valid_path = REPORTS_TBLS / "valid_days.csv"
valid_days = pd.read_csv(valid_path, parse_dates=["date"])
valid_days = valid_days.sort_values("date").reset_index(drop=True)

# --- Map year -> raw CSV path (assumes one file per year, like your current naming) ---
year_files = {}
for p in sorted(DATA_RAW_DIR.glob("*.csv")):
    # find a year token in filename
    y = None
    for token in ["2020","2021","2022","2023","2024","2019"]:
        if token in p.name:
            y = int(token)
            break
    if y:
        year_files[y] = p

# --- Tiny summary so we know our inputs are sane ---
summary = pd.DataFrame.from_dict({
    "valid_days_count": [len(valid_days)],
    "years_present":    [sorted(valid_days["date"].dt.year.unique().tolist())],
    "entry_time":       [ENTRY_T],
    "exit_time":        [EXIT_T],
    "or_window":        [f"{OR_START}–{OR_END} (incl)"],
    "zones(top/bot)":   [f"{TOP_PCT:.2f}/{BOT_PCT:.2f}"],
    "risk(SL/TP pts)":  [f"{SL_PTS}/{TP_PTS}"],
    "$ per point":      [POINT_VAL],
})

print("ROOT:", ROOT)
print("Raw files found by year:")
for y in sorted(year_files):
    print(f"  {y}: {year_files[y].name}")

display(summary.head(1))
display(valid_days.head(5))


ROOT: d:\Projects\OpeningRange
Raw files found by year:
  2020: DAT_ASCII_NSXUSD_M1_2020.csv
  2021: DAT_ASCII_NSXUSD_M1_2021.csv
  2022: DAT_ASCII_NSXUSD_M1_2022.csv
  2023: DAT_ASCII_NSXUSD_M1_2023.csv


,valid_days_count,years_present,entry_time,exit_time,or_window,zones(top/bot),risk(SL/TP pts),$ per point
0,902,"[2020, 2021, 2022, 2023]",10:22,12:00,09:30–10:00 (incl),0.35/0.35,25/75,80.0


,file,date,or_count,or_high,or_low,or_range,trade_count,expected_trade_minutes,missing_minutes,duplicate_minutes,has_entry_1022,has_exit_1200
0,DAT_ASCII_NSXUSD_M1_2020.csv,2020-01-03,31,8822.17,8752.10,70.07,151,151,0,0,True,True
1,DAT_ASCII_NSXUSD_M1_2020.csv,2020-01-06,31,8783.99,8712.49,71.50,151,151,0,0,True,True
2,DAT_ASCII_NSXUSD_M1_2020.csv,2020-01-07,31,8859.59,8816.84,42.75,151,151,0,0,True,True
3,DAT_ASCII_NSXUSD_M1_2020.csv,2020-01-08,31,8879.50,8831.97,47.53,151,151,0,0,True,True
4,DAT_ASCII_NSXUSD_M1_2020.csv,2020-01-09,31,9003.51,8977.64,25.87,151,151,0,0,True,True


### 3.2 — Day loader (returns 09:30–12:00 window and OR slice for a given date)
**What this does:**  
Given a date like `YYYY-MM-DD`, it:
1) Finds the correct year CSV in `data/raw/`.  
2) Parses timestamps (assumes NY local unless `source_timezone` is set).  
3) Slices the **Opening Range** (09:30–10:00 inclusive) and the **trade window** (09:30–12:00 inclusive).  
4) Computes quick QC (missing/duplicate minutes, OR stats, 10:22/12:00 present).  
5) Returns:
   - `win` → DataFrame for **09:30–12:00** (1-min OHLC),
   - `or_slice` → DataFrame for **09:30–10:00**,
   - `qc` → dict with checks and OR stats.

> **How to use (after running the cell):**  
> `win, or_slice, qc = load_day_window("2020-01-03")`


In [ ]:
# 3.2 — Day loader

from pathlib import Path
import pandas as pd
import pytz
import yaml

# -- Paths (work whether you run from notebooks/ or repo root)
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
CONFIG_DIR = ROOT / "config"
DATA_RAW_DIR = ROOT / "data" / "raw"

# -- Load configs
def load_yaml(p: Path):
    with open(p, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)

INSTR    = load_yaml(CONFIG_DIR / "instruments.yml")
STRATEGY = load_yaml(CONFIG_DIR / "strategy.yml")

session   = INSTR.get("session", {})
data_cfg  = INSTR.get("data", {})

TZ_MARKET   = session.get("timezone", "America/New_York")
OR_START    = session.get("or_window", {}).get("start", "09:30")
OR_END      = session.get("or_window", {}).get("end_inclusive", "10:00")  # inclusive
ENTRY_T     = session.get("entry_time", "10:22")
EXIT_T      = session.get("hard_exit_time", "12:00")

DELIM        = data_cfg.get("delimiter", ";")
DATETIME_FMT = data_cfg.get("datetime_format", "%Y%m%d %H%M%S")
SRC_TZ_NAME  = data_cfg.get("source_timezone")  # None ⇒ already NY local

NY = pytz.timezone(TZ_MARKET)
SRC_TZ = pytz.timezone(SRC_TZ_NAME) if SRC_TZ_NAME else None

# -- Map year -> CSV (rebuild if not already built)
year_files = {}
for p in sorted(DATA_RAW_DIR.glob("*.csv")):
    for token in ["2019","2020","2021","2022","2023","2024","2025"]:
        if token in p.name:
            year_files[int(token)] = p
            break

def _parse_index(ts: pd.Series) -> pd.DatetimeIndex:
    idx = pd.to_datetime(ts, format=DATETIME_FMT, errors="coerce")
    if SRC_TZ is not None:
        idx = idx.dt.tz_localize(SRC_TZ, nonexistent="NaT", ambiguous="NaT").dt.tz_convert(NY)
    else:
        idx = idx.dt.tz_localize(NY, nonexistent="NaT", ambiguous="NaT")
    return pd.DatetimeIndex(idx)

def _expected_index_local(day: pd.Timestamp, start_str: str, end_str: str) -> pd.DatetimeIndex:
    start = NY.localize(pd.Timestamp.combine(day.date(), pd.Timestamp(start_str).time()))
    end   = NY.localize(pd.Timestamp.combine(day.date(), pd.Timestamp(end_str).time()))
    return pd.date_range(start=start, end=end, freq="T", tz=NY)

def load_day_window(date_str: str):
    """
    Returns:
      win:      09:30–12:00 inclusive (DataFrame with OHLC/volume)
      or_slice: 09:30–10:00 inclusive
      qc:       dict with OR stats and window integrity checks
    """
    day = pd.to_datetime(date_str).tz_localize(NY)  # local day anchor

    # pick year file
    y = int(day.year)
    if y not in year_files:
        raise FileNotFoundError(f"No CSV file found for year {y} in {DATA_RAW_DIR}")
    fp = year_files[y]

    # read raw csv
    raw = pd.read_csv(fp, sep=DELIM, header=None, dtype=str)
    if raw.shape[1] != 6:
        raise ValueError(f"{fp.name}: expected 6 columns, found {raw.shape[1]}.")

    raw.columns = ["datetime","open","high","low","close","volume"]
    # numeric
    for c in ["open","high","low","close","volume"]:
        raw[c] = pd.to_numeric(raw[c], errors="coerce")

    # index with tz
    idx = _parse_index(raw["datetime"])
    df = raw.drop(columns=["datetime"])
    df.index = idx
    df = df[~df.index.isna()].sort_index()

    # slice this local day
    day_start = NY.localize(pd.Timestamp(day.date()))
    next_day  = day_start + pd.Timedelta(days=1)
    one = df[(df.index >= day_start) & (df.index < next_day)]

    # OR and trade window
    or_slice = one.between_time(OR_START, OR_END, inclusive="both")
    win      = one.between_time("09:30", EXIT_T, inclusive="both")

    # expected minute grid & QC
    tgt_or   = _expected_index_local(day, OR_START, OR_END)
    tgt_win  = _expected_index_local(day, "09:30", EXIT_T)

    missing  = tgt_win.difference(win.index)
    dupes    = int(win.index.duplicated().sum())
    has_1022 = any(win.index.time == pd.Timestamp(ENTRY_T).time())
    has_1200 = any(win.index.time == pd.Timestamp(EXIT_T).time())

    or_high  = float(or_slice["high"].max()) if not or_slice.empty else None
    or_low   = float(or_slice["low"].min())  if not or_slice.empty else None
    or_range = (or_high - or_low) if (or_high is not None and or_low is not None) else None

    qc = {
        "date": day.date().isoformat(),
        "file": fp.name,
        "or_count": int(len(or_slice)),
        "expected_or_minutes": int(len(tgt_or)),
        "or_high": or_high,
        "or_low": or_low,
        "or_range": or_range,
        "trade_count": int(len(win)),
        "expected_trade_minutes": int(len(tgt_win)),
        "missing_minutes": int(len(missing)),
        "duplicate_minutes": dupes,
        "has_entry_1022": bool(has_1022),
        "has_exit_1200": bool(has_1200),
    }

    # Friendly preview
    print(f"[{date_str}] File: {fp.name}")
    print("OR slice 09:30–10:00  (rows):", len(or_slice), " | expected:", len(tgt_or))
    print("Trade win 09:30–12:00 (rows):", len(win),      " | expected:", len(tgt_win))
    print("Has 10:22?", qc["has_entry_1022"], " | Has 12:00?", qc["has_exit_1200"],
          " | Missing:", qc["missing_minutes"], " | Duplicates:", qc["duplicate_minutes"])
    if not or_slice.empty:
        print(f"OR High: {qc['or_high']}  OR Low: {qc['or_low']}  OR Range: {qc['or_range']}")
    print("\nHead (win):")
    display(win.head(5)[["open","high","low","close"]])
    print("Tail (win):")
    display(win.tail(5)[["open","high","low","close"]])

    return win, or_slice, qc


win, or_slice, qc = load_day_window("2020-01-03")


[2020-01-03] File: DAT_ASCII_NSXUSD_M1_2020.csv
OR slice 09:30–10:00  (rows): 31  | expected: 31
Trade win 09:30–12:00 (rows): 151  | expected: 151
Has 10:22? True  | Has 12:00? True  | Missing: 0  | Duplicates: 0
OR High: 8822.17  OR Low: 8752.1  OR Range: 70.06999999999971

Head (win):


C:\Users\Mbaka\AppData\Local\Temp\ipykernel_17840\2422051352.py:56: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return pd.date_range(start=start, end=end, freq="T", tz=NY)


,open,high,low,close
datetime,,,,
2020-01-03 09:30:00-05:00,8752.10,8775.17,8752.10,8774.09
2020-01-03 09:31:00-05:00,8773.68,8791.57,8772.36,8789.39
2020-01-03 09:32:00-05:00,8789.39,8795.81,8783.14,8792.90
2020-01-03 09:33:00-05:00,8793.34,8801.21,8792.40,8800.91
2020-01-03 09:34:00-05:00,8800.91,8806.40,8797.20,8806.39


Tail (win):


,open,high,low,close
datetime,,,,
2020-01-03 11:56:00-05:00,8820.26,8820.26,8815.98,8816.74
2020-01-03 11:57:00-05:00,8816.82,8819.19,8816.45,8816.99
2020-01-03 11:58:00-05:00,8816.92,8817.39,8815.86,8816.46
2020-01-03 11:59:00-05:00,8816.59,8818.00,8816.00,8817.78
2020-01-03 12:00:00-05:00,8817.74,8820.81,8817.17,8820.81


### 3.3 — Signal & barrier levels for one day (deterministic entry at 10:22)

**What this does (read-first):**
- Input: a trading **date** (e.g., `"2020-01-03"`).
- Uses the day loader (3.2) to get:
  - **OR slice** = 09:30–10:00 (inclusive)
  - **Trade window** = 09:30–12:00 (inclusive)
  - **QC** checks
- Computes:
  - `OR_high`, `OR_low`, `OR_range`
  - **Bottom-35% cutoff** = `OR_low + 0.35 * OR_range`
  - **Top-35% cutoff**    = `OR_high − 0.35 * OR_range`
  - **10:22 close** (`P_10:22`)
  - **Decision**: Long / Short / No trade
  - **Barriers** from entry `E = P_10:22`:
    - Long: `SL = E − 25`, `TP = E + 75`
    - Short: `SL = E + 25`, `TP = E − 75`
- Returns a small **summary dict** and shows a neat, human-readable table.

> Note: This cell **only sets the signal & barriers**. The minute-by-minute execution (TP/SL/12:00) comes next (3.4).


In [6]:
# 3.3 — Signal & barrier levels for one day

from dataclasses import dataclass, asdict
from datetime import time
import pandas as pd
import numpy as np

# Expect STRATEGY/INSTR to be in memory from 3.1; fall back safely if not.
try:
    TOP_PCT = STRATEGY["parameters"]["zones"]["top_pct"]
    BOT_PCT = STRATEGY["parameters"]["zones"]["bottom_pct"]
    SL_PTS  = STRATEGY["parameters"]["risk"]["stop_loss_points"]
    TP_PTS  = STRATEGY["parameters"]["risk"]["take_profit_points"]
    ENTRY_T = INSTR["session"]["entry_time"]
    EXIT_T  = INSTR["session"]["hard_exit_time"]
except Exception:
    # Safe defaults matching our baseline
    TOP_PCT = 0.35
    BOT_PCT = 0.35
    SL_PTS  = 25
    TP_PTS  = 75
    ENTRY_T = "10:22"
    EXIT_T  = "12:00"

@dataclass
class DaySignal:
    date: str
    decision: str                 # "long" | "short" | "none" | "no_signal_missing_1022" | "invalid_or"
    entry_time: str
    entry_price: float | None
    sl: float | None
    tp: float | None
    or_high: float | None
    or_low: float | None
    or_range: float | None
    top_cutoff: float | None
    bottom_cutoff: float | None
    has_1022: bool
    has_1200: bool
    notes: str

def _first_close_at(minute_df: pd.DataFrame, hhmm: str) -> float | None:
    """Return close price at a specific HH:MM from a 1-min df indexed by tz-aware timestamps."""
    try:
        target_t = pd.Timestamp(hhmm).time()
        row = minute_df.loc[minute_df.index.time == target_t]
        return float(row["close"].iloc[0]) if not row.empty else None
    except Exception:
        return None

def compute_signal_for_date(date_str: str) -> DaySignal:
    # Use the loader from 3.2
    if "load_day_window" not in globals():
        raise RuntimeError("load_day_window(...) not found. Please run section 3.2 first.")
    win, or_slice, qc = load_day_window(date_str)

    # Basic OR sanity
    if or_slice.empty or qc.get("or_range") is None or qc.get("or_range") <= 0:
        return DaySignal(
            date=date_str, decision="invalid_or", entry_time=ENTRY_T,
            entry_price=None, sl=None, tp=None,
            or_high=qc.get("or_high"), or_low=qc.get("or_low"), or_range=qc.get("or_range"),
            top_cutoff=None, bottom_cutoff=None,
            has_1022=bool(qc.get("has_entry_1022")), has_1200=bool(qc.get("has_exit_1200")),
            notes="Opening range invalid or zero."
        )

    or_high  = float(qc["or_high"])
    or_low   = float(qc["or_low"])
    or_range = float(qc["or_range"])

    # Zone cutoffs
    bottom_cut = or_low  + BOT_PCT * or_range
    top_cut    = or_high - TOP_PCT * or_range

    # Entry price at 10:22
    e = _first_close_at(win, ENTRY_T)
    if e is None:
        return DaySignal(
            date=date_str, decision="no_signal_missing_1022", entry_time=ENTRY_T,
            entry_price=None, sl=None, tp=None,
            or_high=or_high, or_low=or_low, or_range=or_range,
            top_cutoff=top_cut, bottom_cutoff=bottom_cut,
            has_1022=False, has_1200=bool(qc.get("has_exit_1200")),
            notes="No 10:22 bar in trade window."
        )

    # Decision
    if e > top_cut:
        decision = "long"
        sl = e - SL_PTS
        tp = e + TP_PTS
    elif e < bottom_cut:
        decision = "short"
        sl = e + SL_PTS
        tp = e - TP_PTS
    else:
        decision = "none"
        sl = None
        tp = None

    sig = DaySignal(
        date=date_str, decision=decision, entry_time=ENTRY_T,
        entry_price=float(e), sl=float(sl) if sl is not None else None, tp=float(tp) if tp is not None else None,
        or_high=or_high, or_low=or_low, or_range=or_range,
        top_cutoff=float(top_cut), bottom_cutoff=float(bottom_cut),
        has_1022=True, has_1200=bool(qc.get("has_exit_1200")),
        notes="OK"
    )

    # Pretty preview table for humans
    df_preview = pd.DataFrame({
        "date":            [sig.date],
        "decision":        [sig.decision],
        "entry_time":      [sig.entry_time],
        "entry_price":     [sig.entry_price],
        "SL":              [sig.sl],
        "TP":              [sig.tp],
        "OR_low":          [sig.or_low],
        "OR_high":         [sig.or_high],
        "OR_range":        [sig.or_range],
        "bottom_35%":      [sig.bottom_cutoff],
        "top_35%":         [sig.top_cutoff],
        "has_10:22":       [sig.has_1022],
        "has_12:00":       [sig.has_1200],
        "notes":           [sig.notes],
    }).T.rename(columns={0:"value"})
    display(df_preview)

    return sig

# Example (you can change the date):
sig = compute_signal_for_date("2020-01-03")
sig


[2020-01-03] File: DAT_ASCII_NSXUSD_M1_2020.csv
OR slice 09:30–10:00  (rows): 31  | expected: 31
Trade win 09:30–12:00 (rows): 151  | expected: 151
Has 10:22? True  | Has 12:00? True  | Missing: 0  | Duplicates: 0
OR High: 8822.17  OR Low: 8752.1  OR Range: 70.06999999999971

Head (win):


C:\Users\Mbaka\AppData\Local\Temp\ipykernel_17840\2422051352.py:56: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return pd.date_range(start=start, end=end, freq="T", tz=NY)


,open,high,low,close
datetime,,,,
2020-01-03 09:30:00-05:00,8752.10,8775.17,8752.10,8774.09
2020-01-03 09:31:00-05:00,8773.68,8791.57,8772.36,8789.39
2020-01-03 09:32:00-05:00,8789.39,8795.81,8783.14,8792.90
2020-01-03 09:33:00-05:00,8793.34,8801.21,8792.40,8800.91
2020-01-03 09:34:00-05:00,8800.91,8806.40,8797.20,8806.39


Tail (win):


,open,high,low,close
datetime,,,,
2020-01-03 11:56:00-05:00,8820.26,8820.26,8815.98,8816.74
2020-01-03 11:57:00-05:00,8816.82,8819.19,8816.45,8816.99
2020-01-03 11:58:00-05:00,8816.92,8817.39,8815.86,8816.46
2020-01-03 11:59:00-05:00,8816.59,8818.00,8816.00,8817.78
2020-01-03 12:00:00-05:00,8817.74,8820.81,8817.17,8820.81


,value
date,2020-01-03
decision,long
entry_time,10:22
entry_price,8798.09
SL,8773.09
TP,8873.09
OR_low,8752.1
OR_high,8822.17
OR_range,70.07
bottom_35%,8776.6245


DaySignal(date='2020-01-03', decision='long', entry_time='10:22', entry_price=8798.09, sl=8773.09, tp=8873.09, or_high=8822.17, or_low=8752.1, or_range=70.06999999999971, top_cutoff=8797.6455, bottom_cutoff=8776.6245, has_1022=True, has_1200=True, notes='OK')